In [4]:
import numpy as np
import gzip
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interact_manual, widgets,fixed

In [13]:
def generate_biased_rnd_vector_byte(p_in,n_in):
    v_byte_to_ret=np.zeros((n_in,),dtype=np.uint8)
    v_raw_rnd_bin=np.array(np.random.rand(8*n_in)<p_in,dtype=np.uint8)
    for i in range(8):
        v_byte_to_ret+=(v_raw_rnd_bin[i::8]<<i)
    return v_byte_to_ret
def comp_gzip(v_in):
    v_out = np.frombuffer(gzip.compress(v_in),dtype=np.uint8)
    return v_out
def afficher(v_in,nb_bytes_per_lines=128):
    v_in_len=v_in.shape[0]
    nb_lines=int(v_in_len/nb_bytes_per_lines)+1
   
    for j in range(nb_lines):
        s_line='%.3d:'%(j)
        for i in range(nb_bytes_per_lines):
            k=j*nb_bytes_per_lines+i
            if k<v_in_len:
                s_line='%s%.2x'%(s_line,v_in[k])
        print(s_line)

def PlotgzipVsentropy():
    nb_bytes=1024*4
    nb_p=100
    v_p=np.arange(nb_p,dtype=np.float64)/(nb_p-1)
    v_h=np.zeros((nb_p,),dtype=np.float64)
    v_comp_ratio=np.zeros((nb_p,),dtype=np.float64)
    bar = progressbar.ProgressBar(max_value=nb_p)
    for p_i in range(nb_p):
        p=v_p[p_i]
        v_in=generate_biased_rnd_vector_byte(p,nb_bytes)
        v_out = np.frombuffer(gzip.compress(v_in),dtype=np.uint8)
        v_comp_ratio[p_i]=len(v_out)/nb_bytes
        if abs(0.5-p)!=0.5:
            v_h[p_i]=((p)*np.log2(p)+(1-p)*np.log2(1-p))*-1
        bar.update(p_i+1)
    bar.finish()
    fig, ax1 = plt.subplots(figsize=[15,10])
    ax1.set_xlabel('p: proba to generate a 1')
    ax1.set_ylabel('gzip compression ratio', color='tab:red')
    ax1.plot(v_p,v_comp_ratio-v_comp_ratio[0], color='tab:red')
    ax2 = ax1.twinx()
    ax2.plot(v_p,v_h, color='tab:green')
    ax2.set_ylabel('-p.log2(p)-(1-p).log2(1-p)', color='tab:green')

def gzipVsentropy(p=0.5,nb_bytes=1024):
   
    if p*(1-p)==0:
        entropy=0
    else:
        entropy=(np.log2(p)*(p)+np.log2(1-p)*(1-p))/-1

    print('p=%.2f correspond a une entropie de %.2f'%(p,entropy))
   
    v_in=generate_biased_rnd_vector_byte(p,nb_bytes)
    v_comp=comp_gzip(v_in)
    print('\nFICHIER NON COMPRESSE : %d BYTES'%(v_in.shape[0]))
    afficher(v_in,104)
    print('\nFICHIER COMPRESSE : %d BYTES'%(v_comp.shape[0]))
    afficher(v_comp,104)

In [15]:
im=interact(gzipVsentropy)
im.widget.children[0].max=1.0
im.widget.children[0].min=0.0
im.widget.children[0].step=0.01
im.widget.children[1].max=4096
im.widget.children[1].min=32
display(im)

interactive(children=(FloatSlider(value=0.5, description='p', max=1.5, min=-0.5), IntSlider(value=1024, descri…

<function __main__.gzipVsentropy>